# CNN Model - Previous Fire Data and Weather

In [63]:
# Load packages
import boto3
import csv
import io
import json
import math
import numpy as np
import os
import pandas as pd
import pickle
import random

from datetime import datetime as dt
from keras.models import model_from_json
from matplotlib import pyplot as plt
from PIL import Image

### Variables and Hyperparameters

In [22]:
# s3 config
s3_client = boto3.client('s3')
bucket_name = 'hotzone'

# CNN config

# the desired height and width (in pixels) of the matrix to feed into the CNN
# 1 pixel side = 500 meters = 0.310686 miles
matrix_dim = 32

# test size for train/test split
test_size = 0.2

# training epochs
epoc = 10

## Pull Data from S3

In [6]:
def pull_data_from_s3(s3_client, bucket_name, key_name):
    '''
    Pulls pre-processed data from S3.

    Args:
        - s3_client: boto3 s3 client
        - bucket_name: name of bucket on s3 to pull data from
        - key_name: directory/file_name to pull data from
    Returns:
        - Nothing
    
    https://stackoverflow.com/questions/48049557/how-to-write-npy-file-to-s3-directly
    '''
    
    array_data = io.BytesIO()
    s3_client.download_fileobj(bucket_name, key_name, array_data)
    
    array_data.seek(0)
    array = pickle.load(array_data)

    return array

In [7]:
fire_key_name = 'test/fire_2016.pickle'
weather_key_name = 'test/weather_2016.pickle'
label_key_name = 'test/label_2016.pickle'

fire = pull_data_from_s3(s3_client, bucket_name, fire_key_name)
weather = pull_data_from_s3(s3_client, bucket_name, weather_key_name)
Y = pull_data_from_s3(s3_client, bucket_name, label_key_name)

## Build CNN

In [8]:
# import packages

from __future__ import print_function

import tensorflow as tf

import keras
import keras.backend as K

from keras.models import Sequential, Model
from keras.layers import AveragePooling2D, Conv1D, Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Input, concatenate

Using TensorFlow backend.


In [9]:
# compute f1 score manually - taken from https://datascience.stackexchange.com/a/45166

def recall_m(y_true, y_pred):
    '''
    Computes recall.
    
    Args:
        - y_true: true values of target variable.
        - y_pred: predicted values of target variable.
    Returns:
        - recall: true positives / actual results
    '''
    
    true_pos = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_pos = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_pos / (possible_pos + K.epsilon())

    return recall


def precision_m(y_true, y_pred):
    '''
    Computes precision.
    
    Args:
        - y_true: true values of target variable.
        - y_pred: predicted values of target variable.
    Returns:
        - precision: true positives / predicted results
    '''
    
    true_pos = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_pos = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_pos / (predicted_pos + K.epsilon())
    
    return precision


def f1_score(y_true, y_pred):
    '''
    Args:
        - y_true: true values of target variable.
        - y_pred: predicted values of target variable.
    Returns:
        - score: f1 score
    '''
    
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    score = 2*((precision*recall)/(precision+recall+K.epsilon()))
    
    return score

In [10]:
# Create model_2: image data, weather data, and fire speed/direction data with functional API

# Define image inputs shape
image_shape = fire[0].shape
image_inputs = Input(shape = image_shape)

# Define weather inputs shape
weather_shape = weather[0].shape
weather_inputs = Input(shape = weather_shape)

# Add layers for fire image interpretation
fire_1 = AveragePooling2D(pool_size=(2, 2), strides=None, padding='valid')(image_inputs)
fire_2 = Conv2D(32, kernel_size=(3, 3), activation='sigmoid')(fire_1)
fire_3 = Conv2D(64, kernel_size=(3, 3), activation='sigmoid')(fire_2)
fire_4 = MaxPooling2D(pool_size=(2,2), strides=None, padding='valid')(fire_3)
fire_5 = Dropout(0.2)(fire_4)
fire_6 = Flatten()(fire_5)
fire_7 = Dense(64, activation='sigmoid')(fire_6)

# Combine the layers
concat = concatenate([fire_7, weather_inputs])

# Final dense layer 
predictions = Dense(1, activation='sigmoid')(concat)

# Define the model
model_2 = Model(inputs=[image_inputs, weather_inputs], outputs=predictions)

In [11]:
%%time
# compile the model
model_2.compile(
    optimizer='adam', 
    loss='binary_crossentropy', 
    metrics=['accuracy', f1_score, tf.keras.metrics.AUC()]
)

CPU times: user 156 ms, sys: 0 ns, total: 156 ms
Wall time: 155 ms


In [14]:
%%time
# fit the model
model_2.fit(
    x = [fire, weather], 
    y = Y,
    validation_split = test_size, 
    epochs=epoc
)

Train on 5409 samples, validate on 1353 samples
Epoch 1/10
5409/5409 [==============================] - 7s 1ms/step - loss: 0.7074 - accuracy: 0.4981 - f1_score: 0.4063 - auc: 0.4695 - val_loss: 0.7007 - val_accuracy: 0.5137 - val_f1_score: 0.0000e+00 - val_auc: 0.4961
Epoch 2/10
5409/5409 [==============================] - 5s 910us/step - loss: 0.6035 - accuracy: 0.7029 - f1_score: 0.5884 - auc: 0.5515 - val_loss: 0.3852 - val_accuracy: 0.8322 - val_f1_score: 0.7824 - val_auc: 0.6581
Epoch 3/10
5409/5409 [==============================] - 5s 866us/step - loss: 0.2975 - accuracy: 0.8909 - f1_score: 0.8716 - auc: 0.7400 - val_loss: 0.2039 - val_accuracy: 0.9298 - val_f1_score: 0.9191 - val_auc: 0.8017
Epoch 4/10
5409/5409 [==============================] - 5s 926us/step - loss: 0.2030 - accuracy: 0.9338 - f1_score: 0.9296 - auc: 0.8407 - val_loss: 0.1553 - val_accuracy: 0.9342 - val_f1_score: 0.9243 - val_auc: 0.8686
Epoch 5/10
5409/5409 [==============================] - 5s 882us/step 

## Save CNN to S3

In [24]:
saved_model = model_2.to_json()
s3_client.put_object(Body=saved_model, Bucket=bucket_name, Key='models/model_2.json')

{'ResponseMetadata': {'RequestId': 'F3469A306283E0F5',
  'HostId': 'FEVIyr4emxbviWUvAcW6SBUb7yJsPRMn0KZfg2I/p3BMt9F+3+1dEPQVKexCzxCfDGQ5yte44Gw=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'FEVIyr4emxbviWUvAcW6SBUb7yJsPRMn0KZfg2I/p3BMt9F+3+1dEPQVKexCzxCfDGQ5yte44Gw=',
   'x-amz-request-id': 'F3469A306283E0F5',
   'date': 'Sat, 04 Apr 2020 21:57:19 GMT',
   'etag': '"a9002150b4b519d41a00a3afb234fcd5"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"a9002150b4b519d41a00a3afb234fcd5"'}

## Load CNN from S3

In [62]:
s3_client.download_file(bucket_name, 'models/model_2.json', 'model.json')

with open('model.json', 'r') as model_file:
    loaded_model = model_file.read()

model = model_from_json(loaded_model)